In [12]:

import numpy as np
import matplotlib.pyplot as plt


import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import f1_score
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split



import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

import easydict

In [13]:
#데이터 불러오기
train = pd.read_csv('C:\\Users\\user\\Desktop\\computing\\outside\\dacon\\jobcare_recommend\\JobCare_data\\train.csv')
test = pd.read_csv('C:\\Users\\user\\Desktop\\computing\\outside\\dacon\\jobcare_recommend\\JobCare_data\\test.csv')
d_code=pd.read_csv('C:\\Users\\user\\Desktop\\computing\\outside\\dacon\\jobcare_recommend\\Jobcare_data\\속성_D_코드.csv',index_col=0).T.to_dict()
h_code=pd.read_csv('C:\\Users\\user\\Desktop\\computing\\outside\\dacon\\jobcare_recommend\\Jobcare_data\\속성_H_코드.csv',index_col=0).T.to_dict()
l_code=pd.read_csv('C:\\Users\\user\\Desktop\\computing\\outside\\dacon\\jobcare_recommend\\Jobcare_data\\속성_L_코드.csv',index_col=0).T.to_dict()

In [14]:
#데이터 재가공
def add_code(df, d_code, h_code, l_code):
    df = df.copy()   

    # D Code
    df['person_prefer_d_1_n'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_1_s'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_1_m'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_1_l'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['person_prefer_d_2_n'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_2_s'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_2_m'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_2_l'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['person_prefer_d_3_n'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_3_s'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_3_m'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_3_l'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['contents_attribute_d_n'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['contents_attribute_d_s'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['contents_attribute_d_m'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['contents_attribute_d_l'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    # H Code
    df['person_prefer_h_1_m'] = df['person_prefer_h_1'].apply(lambda x: h_code[x]['속성 H 중분류코드'])
    df['person_prefer_h_2_m'] = df['person_prefer_h_2'].apply(lambda x: h_code[x]['속성 H 중분류코드'])
    df['person_prefer_h_3_m'] = df['person_prefer_h_3'].apply(lambda x: h_code[x]['속성 H 중분류코드'])
    df['contents_attribute_h_m'] = df['contents_attribute_h'].apply(lambda x: h_code[x]['속성 H 중분류코드'])

    df['person_prefer_h_1_l'] = df['person_prefer_h_1'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['person_prefer_h_2_l'] = df['person_prefer_h_2'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['person_prefer_h_3_l'] = df['person_prefer_h_3'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['contents_attribute_h_l'] = df['contents_attribute_h'].apply(lambda x: h_code[x]['속성 H 대분류코드'])

    # L Code
    df['contents_attribute_l_n'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 세분류코드'])
    df['contents_attribute_l_s'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 소분류코드'])
    df['contents_attribute_l_m'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 중분류코드'])
    df['contents_attribute_l_l'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 대분류코드'])
    
    return df


train = add_code(train, d_code, h_code, l_code)
test = add_code(test, d_code, h_code, l_code)

In [15]:
train = train.drop(['id', 'contents_open_dt','person_rn', 'contents_rn'], axis=1) 
test = test.drop(['id', 'contents_open_dt','person_rn', 'contents_rn'], axis=1)

# print(train.head())
train['d_l_match_yn']=train['d_l_match_yn'].replace([True,False],[1,0])
train['d_m_match_yn']=train['d_m_match_yn'].replace([True,False],[1,0])
train['d_s_match_yn']=train['d_s_match_yn'].replace([True,False],[1,0])
train['h_l_match_yn']=train['h_l_match_yn'].replace([True,False],[1,0])
train['h_m_match_yn']=train['h_m_match_yn'].replace([True,False],[1,0])
train['h_s_match_yn']=train['h_s_match_yn'].replace([True,False],[1,0])

test['d_l_match_yn']=test['d_l_match_yn'].replace([True,False],[1,0])
test['d_m_match_yn']=test['d_m_match_yn'].replace([True,False],[1,0])
test['d_s_match_yn']=test['d_s_match_yn'].replace([True,False],[1,0])
test['h_l_match_yn']=test['h_l_match_yn'].replace([True,False],[1,0])
test['h_m_match_yn']=test['h_m_match_yn'].replace([True,False],[1,0])
test['h_s_match_yn']=test['h_s_match_yn'].replace([True,False],[1,0])

In [16]:
#컬럼 순서 변경
train.columns
train=train[['d_l_match_yn', 'd_m_match_yn', 'd_s_match_yn', 'h_l_match_yn',
       'h_m_match_yn', 'h_s_match_yn', 'person_attribute_a',
       'person_attribute_a_1', 'person_attribute_b', 'person_prefer_c',
       'person_prefer_d_1', 'person_prefer_d_2', 'person_prefer_d_3',
       'person_prefer_e', 'person_prefer_f', 'person_prefer_g',
       'person_prefer_h_1', 'person_prefer_h_2', 'person_prefer_h_3',
       'contents_attribute_i', 'contents_attribute_a',
       'contents_attribute_j_1', 'contents_attribute_j',
       'contents_attribute_c', 'contents_attribute_k', 'contents_attribute_l',
       'contents_attribute_d', 'contents_attribute_m', 'contents_attribute_e',
       'contents_attribute_h', 'person_prefer_d_1_n',
       'person_prefer_d_1_s', 'person_prefer_d_1_m', 'person_prefer_d_1_l',
       'person_prefer_d_2_n', 'person_prefer_d_2_s', 'person_prefer_d_2_m',
       'person_prefer_d_2_l', 'person_prefer_d_3_n', 'person_prefer_d_3_s',
       'person_prefer_d_3_m', 'person_prefer_d_3_l', 'contents_attribute_d_n',
       'contents_attribute_d_s', 'contents_attribute_d_m',
       'contents_attribute_d_l', 'person_prefer_h_1_m', 'person_prefer_h_2_m',
       'person_prefer_h_3_m', 'contents_attribute_h_m', 'person_prefer_h_1_l',
       'person_prefer_h_2_l', 'person_prefer_h_3_l', 'contents_attribute_h_l',
       'contents_attribute_l_n', 'contents_attribute_l_s',
       'contents_attribute_l_m', 'contents_attribute_l_l','target']]

test=test[['d_l_match_yn', 'd_m_match_yn', 'd_s_match_yn', 'h_l_match_yn',
       'h_m_match_yn', 'h_s_match_yn', 'person_attribute_a',
       'person_attribute_a_1', 'person_attribute_b', 'person_prefer_c',
       'person_prefer_d_1', 'person_prefer_d_2', 'person_prefer_d_3',
       'person_prefer_e', 'person_prefer_f', 'person_prefer_g',
       'person_prefer_h_1', 'person_prefer_h_2', 'person_prefer_h_3',
       'contents_attribute_i', 'contents_attribute_a',
       'contents_attribute_j_1', 'contents_attribute_j',
       'contents_attribute_c', 'contents_attribute_k', 'contents_attribute_l',
       'contents_attribute_d', 'contents_attribute_m', 'contents_attribute_e',
       'contents_attribute_h', 'person_prefer_d_1_n',
       'person_prefer_d_1_s', 'person_prefer_d_1_m', 'person_prefer_d_1_l',
       'person_prefer_d_2_n', 'person_prefer_d_2_s', 'person_prefer_d_2_m',
       'person_prefer_d_2_l', 'person_prefer_d_3_n', 'person_prefer_d_3_s',
       'person_prefer_d_3_m', 'person_prefer_d_3_l', 'contents_attribute_d_n',
       'contents_attribute_d_s', 'contents_attribute_d_m',
       'contents_attribute_d_l', 'person_prefer_h_1_m', 'person_prefer_h_2_m',
       'person_prefer_h_3_m', 'contents_attribute_h_m', 'person_prefer_h_1_l',
       'person_prefer_h_2_l', 'person_prefer_h_3_l', 'contents_attribute_h_l',
       'contents_attribute_l_n', 'contents_attribute_l_s',
       'contents_attribute_l_m', 'contents_attribute_l_l']]


In [17]:
torch.manual_seed(42)


train.shape
test.shape

(46404, 58)

In [18]:

x = train.iloc[0:10000, :-1]
y = train.iloc[0:10000, -1]

x= x.to_numpy()
print(x[0])
print(y[0])
x=np.where(x > 1000, x/1000, x)
x=np.where(x > 100, x/100, x)
x=np.where(x > 10, x/10, x)
# x=x/100
x=torch.Tensor(x)
print(x.size())
print(x[0][1])
y=torch.Tensor(y)
# print(y)

x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, shuffle=True, stratify=y, random_state=42)



# print(x)
# print(y)


batch_size=32

dataset_train = TensorDataset(x_train, y_train)
dataloader_train = DataLoader(dataset_train, batch_size, shuffle=True)


dataset_valid = TensorDataset(x_valid, y_valid)
dataloader_valid = DataLoader(dataset_valid, batch_size, shuffle=True)




x_test=test.iloc[0:1000]
x_test=x_test.to_numpy()
x_test=np.where(x_test > 1000, x_test/1000, x_test)
x_test=np.where(x_test > 100, x_test/100, x_test)
x_test=np.where(x_test > 10, x_test/10, x_test)
x_test=x_test/100

print(x_test.shape)
x_test=torch.FloatTensor(x_test)

[   1    1    1    0    0    0    1    4    3    5  275  370  369    8
    1    1    4   95   59    3    3   10    2    1    2 1608  275    1
    4  139  275  274  274  216  369  368  297  216  369  368  297  216
  275  274  274  216  316  398  368  422    3   94   58   94 1607 1606
 1605 2016]
1
torch.Size([10000, 58])
tensor(1.)
(1000, 58)


In [19]:



class LogisticRegression(nn.Module):
    
    def __init__(self):
        super(LogisticRegression, self).__init__()

        # self.linear = nn.Sequential(
        self.linear1=nn.Linear(58,32)
        self.batch_normal1=nn.BatchNorm1d(32)
        self.LeakRelu=nn.LeakyReLU()

        self.linear2=nn.Linear(32, 32)
        self.batch_normal2=nn.BatchNorm1d(32)
        self.LeakRelu=nn.LeakyReLU()

        self.linear3=nn.Linear(32, 16)
        self.batch_normal3=nn.BatchNorm1d(16)
        self.LeakRelu=nn.LeakyReLU()

        self.linear4=nn.Linear(16, 1)
        self.batch_normal4=nn.BatchNorm1d(1)
        self.LeakRelu=nn.LeakyReLU()

        self.sigmoid=nn.Sigmoid()    
        torch.nn.init.xavier_uniform_(self.linear1.weight)
        torch.nn.init.xavier_uniform_(self.linear2.weight)
        torch.nn.init.xavier_uniform_(self.linear3.weight)
        torch.nn.init.xavier_uniform_(self.linear4.weight)
        # )        

        self.model=nn.Sequential(self.linear1, self.batch_normal1, self.LeakRelu, self.linear2,self.batch_normal2, self.LeakRelu, self.linear3,self.batch_normal3,self.LeakRelu,
        self.linear4, self.batch_normal4,self.LeakRelu,self.sigmoid)

    def forward(self, x):
        return self.model(x)

model1=LogisticRegression()

In [20]:
optimizer1 = optim.Adam(model1.parameters(), lr=0.001)
print(model1.parameters)


<bound method Module.parameters of LogisticRegression(
  (linear1): Linear(in_features=58, out_features=32, bias=True)
  (batch_normal1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (LeakRelu): LeakyReLU(negative_slope=0.01)
  (linear2): Linear(in_features=32, out_features=32, bias=True)
  (batch_normal2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear3): Linear(in_features=32, out_features=16, bias=True)
  (batch_normal3): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear4): Linear(in_features=16, out_features=1, bias=True)
  (batch_normal4): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (sigmoid): Sigmoid()
  (model): Sequential(
    (0): Linear(in_features=58, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Linear

In [21]:
nb_epochs = 100
accuracy_list=[]

model1.train()
for epoch in range(nb_epochs + 1):
    for batch_idx, samples in enumerate(dataloader_train):

        x_train, y_train = samples

        x_train=x_train
        y_train=y_train


        # H(x) 계산
        hypothesis = model1(x_train)
        
        
    
        # cost 계산
        # print(y_train.size())
        hypothesis=hypothesis.squeeze()
        # print(hypothesis.size())
        # print(hypothesis)
        # print(y_train)
        cost = F.binary_cross_entropy(hypothesis, y_train)

        # cost로 H(x) 개선
        optimizer1.zero_grad()
        cost.backward()
        optimizer1.step()

        bigo=torch.FloatTensor([0.5])
        

        # 20번마다 로그 출력
        if epoch % 10 == 0:
          if batch_idx %100 == 0:
              prediction = hypothesis >= bigo # 예측값이 0.5를 넘으면 True로 간주
              correct_prediction = prediction.float() == y_train # 실제값과 일치하는 경우만 True로 간주
              accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산
              f1 = f1_score(y_train.cpu(), prediction.float().cpu())
              print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f} Accuracy {:2.2f}%  f1 score : {:2.6f}'.format( # 각 에포크마다 정확도를 출력
                  epoch, nb_epochs, batch_idx+1, len(dataloader_train), cost.item(), accuracy * 100, f1
              ))

    torch.save(model1.state_dict(), f'{epoch:04d}.pt')







Epoch    0/100 Batch 1/250 Cost: 0.785893 Accuracy 40.62%  f1 score : 0.457143
Epoch    0/100 Batch 101/250 Cost: 0.687099 Accuracy 46.88%  f1 score : 0.585366
Epoch    0/100 Batch 201/250 Cost: 0.683651 Accuracy 68.75%  f1 score : 0.736842
Epoch   10/100 Batch 1/250 Cost: 0.702255 Accuracy 56.25%  f1 score : 0.222222
Epoch   10/100 Batch 101/250 Cost: 0.675796 Accuracy 53.12%  f1 score : 0.285714
Epoch   10/100 Batch 201/250 Cost: 0.690203 Accuracy 68.75%  f1 score : 0.166667
Epoch   20/100 Batch 1/250 Cost: 0.688154 Accuracy 68.75%  f1 score : 0.375000
Epoch   20/100 Batch 101/250 Cost: 0.665947 Accuracy 75.00%  f1 score : 0.500000
Epoch   20/100 Batch 201/250 Cost: 0.670840 Accuracy 71.88%  f1 score : 0.307692
Epoch   30/100 Batch 1/250 Cost: 0.654789 Accuracy 71.88%  f1 score : 0.470588
Epoch   30/100 Batch 101/250 Cost: 0.661993 Accuracy 75.00%  f1 score : 0.333333
Epoch   30/100 Batch 201/250 Cost: 0.738954 Accuracy 59.38%  f1 score : 0.315789
Epoch   40/100 Batch 1/250 Cost: 0.6

In [22]:
# 테스트 데이터를 사용하여 모델을 테스트한다.
with torch.no_grad(): # torch.no_grad()를 하면 gradient 계산을 수행하지 않는다.
    X_valid = x_valid
    Y_valid = y_valid
    model1.eval()
    prediction = model1(X_valid)

    print(prediction)

    predictions = prediction >= bigo # 예측값이 0.5를 넘으면 True로 간주
    
    # print(predictions)
    # print('='*20)
    # print(predictions.size())
    # print('='*20)
    # print(predictions)
    # print(predictions.shape)
    Y_valid=torch.unsqueeze(Y_valid,1)
    print(Y_valid.shape)

    correct_prediction = predictions == Y_valid # 실제값과 일치하는 경우만 True로 간주

    # print(y_valid)

    # correct_prediction=predictions.replace([True,False],[1,0])

    # print(correct_prediction)
    # print(correct_prediction.size())
    # print(len(correct_prediction))

    accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산

    print('Accuracy:', accuracy)


tensor([[0.4948],
        [0.4939],
        [0.4963],
        ...,
        [0.4999],
        [0.6048],
        [0.4960]])
torch.Size([2000, 1])
Accuracy: 0.62


In [23]:
result=pd.read_csv('C:\\Users\\user\\Desktop\\computing\\outside\\dacon\\jobcare_recommend\\Jobcare_data\\sample_submission.csv')


hypothesis=model1(x_test)

prediction = hypothesis >= torch.FloatTensor([0.5]) # 예측값이 0.5를 넘으면 True로 간주
prediction = torch.squeeze(prediction)
prediction=pd.Series(prediction)
print(prediction)
prediction=prediction.replace([True,False],[1,0])

result['target']=prediction
result.to_csv('result.csv',index=False)



0      True
1      True
2      True
3      True
4      True
       ... 
995    True
996    True
997    True
998    True
999    True
Length: 1000, dtype: bool


In [39]:
model=LogisticRegression()

model.load_state_dict(torch.load('C:\\Users\\user\\Desktop\\computing\\outside\\dacon\\jobcare_recommend\\0040.pt'))
model.eval()

result=model(X_valid)
result1 = result >= bigo # 예측값이 0.5를 넘으면 True로 간주
correct_prediction1 = result1 == Y_valid

accuracy1 = correct_prediction1.sum().item() / len(correct_prediction) # 정확도를 계산

print('Accuracy:', accuracy1)

Accuracy: 0.6255
